# Algorithmic Fairness, Accountability, and Ethics, Spring 2024

## Mandatory Assignment 1

Please use the following code to prepare the dataset.
 

In [50]:
from folktables.acs import adult_filter
from folktables import ACSDataSource
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["CA"], download=True)

feature_names = ['AGEP', # Age
                 "CIT", # Citizenship status
                 'COW', # Class of worker
                 "ENG", # Ability to speak English
                 'SCHL', # Educational attainment
                 'MAR', # Marital status
                 "HINS1", # Insurance through a current or former employer or union
                 "HINS2", # Insurance purchased directly from an insurance company
                 "HINS4", # Medicaid
                 "RAC1P", # Recoded detailed race code
                 'SEX']

target_name = "PINCP" # Total person's income

def data_processing(data, features, target_name:str, threshold: float = 35000):
    df = data
    ### Adult Filter (STARTS) (from Foltktables)
    df = df[~df["SEX"].isnull()]
    df = df[~df["RAC1P"].isnull()]
    df = df[df['AGEP'] > 16]
    df = df[df['PINCP'] > 100]
    df = df[df['WKHP'] > 0]
    df = df[df['PWGTP'] >= 1]
    ### Adult Filter (ENDS)
    ### Groups of interest
    sex = df["SEX"].values
    ### Target
    df["target"] = df[target_name] > threshold
    target = df["target"].values
    df = df[features + ["target", target_name]] ##we want to keep df before one_hot encoding to make Bias Analysis
    df_processed = df[features].copy()
    cols = [ "HINS1", "HINS2", "HINS4", "CIT", "COW", "SCHL", "MAR", "SEX", "RAC1P"]
    df_processed = pd.get_dummies(df_processed, prefix=None, prefix_sep='_', dummy_na=False, columns=cols, drop_first=True)
    df_processed = pd.get_dummies(df_processed, prefix=None, prefix_sep='_', dummy_na=True, columns=["ENG"], drop_first=True)
    return df_processed, df, target, sex

data, data_original, target, group = data_processing(acs_data, feature_names, target_name)

X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
    data, target, group, test_size=0.2, random_state=0)

In [51]:
## Imports
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import StandardScaler 

## Task 1 (Classifiers and fairness considerations)  
1. Starting from the template, train two different classifiers on the training data: a white-box 
model using logistic regression, and a black-box model using a random forest. Consider 
feature engineering and scaling steps necessary for some of these classifiers and summarize 
the necessary changes in your report. For both models, report on the accuracy of the 
classifier on the test set.
2. For each classifier, measure statistical parity, equalized odds (both in terms of  T = 0  and  T 
= 1 ), and equality of outcome (both in terms of  S = 0  and  S = 1 ) (Lecture 2). Plot the 
results and discuss the differences that you observe.  
3. Change the classification pipeline to (approximately) fulfill one of the fairness criteria by post-
processing the results. How did the intervention influence the different fairness criteria, how 
did it change the accuracy of the classification?

In [52]:
## Constants

seed = 23

In [54]:
column_trans = ColumnTransformer(
    [('scaler', StandardScaler(),['AGEP'])], 
    remainder='passthrough')

column_trans.fit(X_train)


X_train.AGEP = column_trans.transform(X_train)[:,0]

X_test.AGEP = column_trans.transform(X_test)[:,0]


LRclf = LogisticRegression(max_iter=5000, penalty='l2', C=0.98497534359086438, tol=1e-4, solver='saga', random_state=seed)
LRclf.fit(X_train, y_train)
LRclf.score(X_test, y_test)

0.7696317685840595

In [57]:
#Run this cell to use categorical features instead
data_original.loc[data_original[data_original.ENG.isna()].index, 'ENG'] = -1
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
    data_original[['AGEP', 'CIT', 'COW', 'SCHL', 'MAR', 'HINS1', 'HINS2', 'HINS4', 'RAC1P',
       'ENG', 'SEX']], target, group, test_size=0.2, random_state=0)

In [58]:
RFclf = RandomForestClassifier(max_depth=2, random_state=seed, n_jobs=-1)
RFclf.fit(X_train, y_train)
RFclf.score(X_test, y_test)

0.757851429739606

## Task 2 (Explaining white-box models)  
1. Explain the trained logistic regression model. In particular, discuss which features in the 
model are deemed most relevant. Reflect on the interpretation. Does it fit your intuition 
about the prediction task?
2. Pick one data point in the test dataset. Find a counterfactual data point that contrasts the 
outcome of the inference on this data point (e.g., "had X had feature P >=, then it had been 
classified as ..."). Describe how you used the model explanation to find such a counterfactual.


In [56]:
weights = {}
for i in range(len(LRclf.coef_[0])):
    weights[LRclf.feature_names_in_[i]] = [LRclf.coef_[0][i]]

for k,v in weights.items():
    weights[k].append(np.exp(v)[0])

sorted(weights.items(), key=lambda x: x[1], reverse=True)

[('SCHL_24.0', [2.4623154305875823, 11.731944618900648]),
 ('SCHL_23.0', [2.4337399113725606, 11.401442829379475]),
 ('SCHL_22.0', [2.2454122911649703, 9.444308558858031]),
 ('SCHL_21.0', [1.7451835119245798, 5.726952343041435]),
 ('SCHL_20.0', [0.9972422337533567, 2.7107957697281617]),
 ('HINS4_2', [0.8352327936304097, 2.305350656773828]),
 ('SCHL_19.0', [0.7108751261395431, 2.0357720366538357]),
 ('SCHL_18.0', [0.6628898640449583, 1.9403917080210855]),
 ('SCHL_17.0', [0.6210348419585144, 1.8608527342863883]),
 ('COW_5.0', [0.6084806863025892, 1.8376373292198678]),
 ('SCHL_3.0', [0.5713887140797329, 1.7707243746495454]),
 ('AGEP', [0.5571323241610381, 1.745659330573422]),
 ('SCHL_2.0', [0.46410058456358116, 1.590582950486057]),
 ('SCHL_16.0', [0.4485168544155348, 1.565987874269228]),
 ('COW_7.0', [0.2674562260973818, 1.306636432008279]),
 ('SCHL_12.0', [0.2512156255843019, 1.285587259951142]),
 ('SCHL_15.0', [0.2137259900227065, 1.2382833063224938]),
 ('SCHL_10.0', [0.1848394987824205

In [73]:
RF_imps = sorted([[round(i,3), j] for i, j in zip(RFclf.feature_importances_, RFclf.feature_names_in_)], reverse=True)

RF_imps

[[0.331, 'AGEP'],
 [0.229, 'SCHL'],
 [0.168, 'HINS1'],
 [0.137, 'HINS4'],
 [0.079, 'MAR'],
 [0.028, 'ENG'],
 [0.014, 'CIT'],
 [0.006, 'SEX'],
 [0.006, 'RAC1P'],
 [0.002, 'COW'],
 [0.001, 'HINS2']]

## Task 3 (Model-agnostic explanations)  
1. Both for the white-box and the black-box classifier, use the  shap  module to explain 
predictions. Contrast the two models to each other: What are similarities, how do they differ?
2. For logistic regression, compare the model-agnostic explanation to your analysis in Task 2. 
How do the explanations differ?

## Task 4 (Reflection)  
Given the outcome of your study, which classifier is most suited for the prediction task under 
accuracy, explainability, and fairness considerations?